In [22]:
import os

thermal_dir = "thermal_images"
for root, dirs, files in os.walk(thermal_dir):
    for f in files:
        if(f[-3:] != "csv"): 
            os.remove(os.path.join(root, f))

In [2]:
import pandas as pd 
import numpy as np
from pandas import DataFrame

timestamp = pd.read_csv("project_data/thermal_timestamps.csv")

timestamp.drop(timestamp.columns[-2:], axis=1, inplace=True)
timestamp.drop(timestamp.index[[0,3]], axis=0, inplace=True)

start_mins = [1, 5, 9]
end_mins = [3, 7, 11]
start_sec = [2, 6, 10]
end_sec = [4, 8, 12]

sec_timestamp = DataFrame(columns=["s1", "e1", "s2", "e2", "s3", "e3"])
for i in timestamp.index.values:
    values = timestamp.loc[i].values.astype(int)
    s1, s2, s3 = values[start_mins]*60 + values[start_sec]
    e1, e2, e3 = values[end_mins]*60 + values[end_sec]
    sec_timestamp.loc[values[0]] = [s1, e1, s2, e2, s3, e3]
sec_timestamp.to_csv("project_data/thermal_timestamps.csv")

IndexError: index 5 is out of bounds for axis 1 with size 5

In [83]:
timestamps = pd.read_csv("project_data/thermal_timestamps.csv", index_col=0)
labels = pd.read_csv("project_data/labels.csv", usecols=[0,1], dtype={"id": int, "label": str}, index_col=0)
# labels
def get_story_by_frame(subject, frame):
    second= frame/10.0 if (subject<15) else frame
    sub_data = timestamps.loc[subject].values
    i, j = 0, 1
    story = -1
    for k in range(3):
        if (sub_data[i] <= second <= sub_data[j]):
            story = k+1
            break
        i += 2
        j += 2
    return story
    
    
def get_label_by_frame(subject, frame):
    story = get_story_by_frame(subject, frame)
    if (story == -1): return None
    return labels.loc[subject].values[0][story-1]

timestamps

,s1,e1,s2,e2,s3,e3
1,9,33,41,59,78,83
2,140,157,157,184,184,237
4,62,77,100,107,146,154
5,156,195,199,246,254,294
6,75,97,102,126,150,167
7,174,227,243,294,313,363
8,80,123,130,170,170,217
9,109,131,131,145,145,157
10,216,223,230,249,325,335
11,159,218,306,357,361,404


In [ ]:
def plot_bins(bin_places):
    cm = plt.cm.get_cmap('plasma')
    n, bins, patches = plt.hist(frame_1, bins=bin_places[1], color='green')
    bin_centers = 0.5 * (bins[:-1] + bins[1:])
    col = bin_centers - min(bin_centers)
    col /= max(col)
    for c, p in zip(col, patches):
        plt.setp(p, 'facecolor', cm(c))
    plt.savefig("temp.png", dpi=1000)
    plt.show()

def save_as_image(image_matrix, name="temp.png"):
    oldmin, oldmax= image_matrix.min(), image_matrix.max()
    newmin, newmax= 0, 255
    image_matrix = (((image_matrix-oldmin)*(newmax-newmin))/(oldmax-oldmin)) + newmin
    image_matrix = map(lambda arr: map(lambda x: int(x), arr), image_matrix)
    png.fromarray(image_matrix, 'L').save(name)

In [4]:
import pandas as pd 
import numpy as np
import pickle
from pandas import DataFrame
from matplotlib import pyplot as plt
import os

thermal_folder = "thermal_images/"
timestamp_folder = "project_data/thermal_timestamps.csv"
label_folder = "project_data/labels.csv"

filter_temp = 32

timestamps = pd.read_csv(timestamp_folder, index_col=0)
labels = pd.read_csv(label_folder, usecols=[0,1], dtype={"id": int, "label": str}, index_col=0)
game = pd.read_csv(label_folder, usecols=[0,1], dtype={"id": int, "game": int}, index_col=0)
# print timestamps.loc[10]

def get_story_by_frame(subject, frame):
    second= frame/10.0 if (subject<15) else frame
    sub_data = timestamps.loc[subject].values
    if (second < sub_data[0]): return 0
    i, j = 0, 1
    story = -1
    for k in range(3):
        if (sub_data[i] <= second <= sub_data[j]):
            story = k+1
            break
        i += 2
        j += 2
    return story

def get_label_by_frame(subject, frame):
    story = get_story_by_frame(subject, frame)
    if (story == 0 or story == -1): return (story, -1)
    return (story, labels.loc[subject].values[0][story-1])

def frame_with_info_traverse(thermal_path):
    walker = os.walk(thermal_path)
    walker.next() # skip the main dir
    for root, dirs, files in walker:
        if (len(files) == 0): continue # if the folder is empty or is root folder
        subject = int(root.split("/")[-2].strip("Subject")) # thermal_images/Subject10/CSV images => 10
        for f in files:
            frame_nr = int(f.split("_")[1].strip(".csv")) # FL-000020_369.csv => 369
            story, label = get_label_by_frame(subject, frame_nr) # get the story number and label of the frame
            if (story == -1): continue # skip if the frame is between the stories
            csv_frame = pd.read_csv(os.path.join(root, f)).as_matrix() # read the csv data
            yield (subject, story, frame_nr, csv_frame, label)

def extract_frame_features(frame_1):
    frame_1[frame_1 < filter_temp] = 0 #filter all pixels lower than 30
    frame_1 = frame_1[np.where(frame_1 != 0)].flatten() # new array with only filtered values
    features = np.array([frame_1.min(), frame_1.max(), np.mean(frame_1)]) # get mean, max and min values of pixels
    features = np.append(features, features[1] - features[0]) # add max difference in pixels
    bin_places = np.histogram(frame_1, np.linspace(filter_temp, 37.0, 101))[0] # take the values of the bins
#     plot_bins(bin_places) # plot the bins if needed
    features = np.concatenate((features, bin_places)) # add primary features to bin features
    return features


csv_generator = frame_with_info_traverse(thermal_folder)
baseline_count = dict(zip(range(1, 25), np.zeros(24)))
baseline = dict(zip(range(1, 25), np.zeros(24)))
control_group = {}
test_group = {}
c = 0
for subject, story, fnr, frame, label in csv_generator:
    print (subject, fnr)
    c +=1
    if (c == 300): break
    features = extract_frame_features(frame)
    if (story==0):
        baseline_count[subject] = baseline_count[subject] + 1
        baseline[subject] = baseline[subject] + features
        continue
    update_dict = control_group if (game.loc[subject].values[0]==0) else test_group
    if ((subject, story) not in update_dict): 
        update_dict[(subject, story)] = np.array([features, label])
    else:
        update_dict[(subject, story)] = np.vstack((update_dict[(subject, story)], [features, label]))


for key in baseline.keys():
    if baseline_count[key]==0: continue
    baseline[key] = baseline[key]/baseline_count[key]

def serialize(group, fi):
    with open(fi, 'wb') as f:
        pickle.dump(group, f)

serialize(control_group, "control_group_data.pickle")
serialize(test_group, "test_group.pickle")
serialize(baseline, "baseline.pickle")


(10, 369)
(10, 37)
(10, 370)
(10, 371)
(10, 372)
(10, 373)
(10, 374)
(10, 375)
(10, 376)
(10, 377)
(10, 378)
(10, 4)
(10, 414)
(10, 43)
(10, 445)
(10, 45)
(10, 470)
(10, 471)
(10, 472)
(10, 473)
(10, 474)
(10, 475)
(10, 476)
(10, 477)
(10, 478)
(10, 479)
(10, 48)
(10, 480)
(10, 481)
(10, 482)
(10, 483)
(10, 484)
(10, 451)
(10, 452)
(10, 453)
(10, 454)
(10, 455)
(10, 456)
(10, 457)
(10, 458)
(10, 459)
(10, 46)
(10, 460)
(10, 461)
(10, 462)
(10, 463)
(10, 464)
(10, 465)
(10, 466)
(10, 467)
(10, 468)
(10, 469)
(10, 486)
(10, 487)
(10, 488)
(10, 489)
(10, 49)
(10, 490)
(10, 491)
(10, 492)
(10, 493)
(10, 494)
(10, 495)
(10, 496)
(10, 497)
(10, 498)
(10, 499)
(10, 5)
(10, 1029)
(10, 1044)
(10, 106)
(10, 1075)
(10, 1090)
(10, 1105)
(10, 1146)
(10, 1161)
(10, 1177)
(10, 1192)
(10, 1207)
(10, 1222)
(10, 1238)
(10, 500)
(10, 501)
(10, 502)
(10, 503)
(10, 504)
(10, 505)
(10, 506)
(10, 507)
(10, 508)
(10, 509)
(10, 51)
(10, 510)
(10, 511)
(10, 512)
(10, 513)
(10, 514)
(10, 516)
(10, 517)
(10, 518)

In [7]:
def deserialize(f):
    with open(f, 'rb') as fil:
        return pickle.load(fil)

baseline = deserialize("baseline.pickle")
baseline

{1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: array([  3.20036900e+01,   3.51235482e+01,   3.33830720e+01,
          3.11985825e+00,   1.38975000e+02,   1.59775000e+02,
          1.65275000e+02,   1.59350000e+02,   1.80525000e+02,
          1.89025000e+02,   1.85250000e+02,   2.20075000e+02,
          2.27875000e+02,   2.40050000e+02,   2.18300000e+02,
          2.33000000e+02,   2.25900000e+02,   1.96600000e+02,
          2.19725000e+02,   2.16225000e+02,   2.11950000e+02,
          1.90950000e+02,   2.11000000e+02,   2.18050000e+02,
          1.99550000e+02,   2.21100000e+02,   2.25900000e+02,
          2.32850000e+02,   2.12375000e+02,   2.44475000e+02,
          2.58175000e+02,   2.63400000e+02,   2.65175000e+02,
          2.47850000e+02,   2.85175000e+02,   2.96925000e+02,
          3.19825000e+02,   3.24725000e+02,   3.13725000e+02,
          3.27500000e+02,   3.24725000e+02,   2.99500000e+02,
          2.98400000e+02,   2.62850000e+02,   2

In [18]:
import os 
walker = os.walk("project_data")
for i in range(2):
    print walker.next()

('project_data', [], ['labels.csv', 'thermal_timestamps.csv'])


StopIteration: 